<a href="https://colab.research.google.com/github/Alexiagmz/Regresi-n-Logistica/blob/main/M3_Actividad_4_(Regresi%C3%B3n_Log%C3%ADstica)_Barcelona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cargamos librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [54]:
#Carga desde un archivo .csv sin indice y eliminar filas innecesarias
barcelona=pd.read_csv("Barcelona_limpios_valores.csv")
barcelona.head(5)

,Unnamed: 0,id,host_id,host_total_listings_count,accommodates,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,...,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,bathrooms,bedrooms,amenities,calendar_last_scraped
0,0,269467,1411775,7.0,2.0,176.0,7.0,760.0,7.0,7.0,...,t,t,la Sagrada Família,Eixample,Entire loft,Entire home/apt,1.0,1.0,"[""Kitchen"", ""Patio or balcony"", ""Host greets y...",2024-06-15
1,1,270197,1414702,112.0,6.0,74.0,32.0,760.0,32.0,32.0,...,t,t,el Raval,Ciutat Vella,Entire rental unit,Entire home/apt,1.5,4.0,"[""Kitchen"", ""Washer"", ""Smoking allowed"", ""Wifi...",2024-06-15
2,2,18674,71615,45.0,3.0,257.0,1.0,1125.0,1.0,4.0,...,t,t,la Sagrada Família,Eixample,Entire rental unit,Entire home/apt,2.0,3.0,"[""Pack \u2019n play/Travel crib"", ""Free street...",2024-06-15
3,3,272282,1425070,2.0,3.0,136.0,4.0,1125.0,3.0,3.0,...,t,t,l'Antiga Esquerra de l'Eixample,Eixample,Entire rental unit,Entire home/apt,Unknown,3.0,"[""Pack \u2019n play/Travel crib"", ""Kitchen"", ""...",2024-06-15
4,4,23197,90417,13.0,5.0,249.0,3.0,32.0,1.0,7.0,...,t,t,el Besòs i el Maresme,Sant Martí,Entire rental unit,Entire home/apt,2.0,3.0,"[""Free street parking"", ""Neutral body soap"", ""...",2024-06-15


In [100]:
# Reemplazar 'unknown' por NaN
barcelona['bathrooms'] = barcelona['bathrooms'].replace('Unknown', np.nan)
barcelona['bedrooms'] = barcelona['bedrooms'].replace('Unknown', np.nan)
# convertir a float
barcelona['bathrooms'] = barcelona['bathrooms'].astype(float)
barcelona['bedrooms'] = barcelona['bedrooms'].astype(float)

In [101]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelona= barcelona.fillna(method='ffill')

<ipython-input-101-c897af8e098f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-101-c897af8e098f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='ffill')


In [102]:
barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18923 entries, 0 to 18922
Data columns (total 45 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    18923 non-null  int64  
 1   id                            18923 non-null  int64  
 2   host_id                       18923 non-null  int64  
 3   host_total_listings_count     18923 non-null  float64
 4   accommodates                  18923 non-null  float64
 5   price                         18923 non-null  float64
 6   minimum_nights                18923 non-null  float64
 7   maximum_nights                18923 non-null  float64
 8   minimum_minimum_nights        18923 non-null  float64
 9   maximum_minimum_nights        18923 non-null  float64
 10  minimum_maximum_nights        18923 non-null  float64
 11  maximum_maximum_nights        18923 non-null  float64
 12  minimum_nights_avg_ntm        18923 non-null  float64
 13  m

Regresión logística

**1. **

In [144]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['price'].max()
Min=barcelona['price'].min()
Limites= [Min, Max]
Limites

[10.0, 414.0]

In [145]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(10.0, 414.0, 150)
intervalos

array([ 10.        ,  12.7114094 ,  15.42281879,  18.13422819,
        20.84563758,  23.55704698,  26.26845638,  28.97986577,
        31.69127517,  34.40268456,  37.11409396,  39.82550336,
        42.53691275,  45.24832215,  47.95973154,  50.67114094,
        53.38255034,  56.09395973,  58.80536913,  61.51677852,
        64.22818792,  66.93959732,  69.65100671,  72.36241611,
        75.0738255 ,  77.7852349 ,  80.4966443 ,  83.20805369,
        85.91946309,  88.63087248,  91.34228188,  94.05369128,
        96.76510067,  99.47651007, 102.18791946, 104.89932886,
       107.61073826, 110.32214765, 113.03355705, 115.74496644,
       118.45637584, 121.16778523, 123.87919463, 126.59060403,
       129.30201342, 132.01342282, 134.72483221, 137.43624161,
       140.14765101, 142.8590604 , 145.5704698 , 148.28187919,
       150.99328859, 153.70469799, 156.41610738, 159.12751678,
       161.83892617, 164.55033557, 167.26174497, 169.97315436,
       172.68456376, 175.39597315, 178.10738255, 180.81

In [146]:
#Creamos las categorías
categorias= ["barato", "caro"]

In [148]:
#Finalmente creamos las categorías en la columna numérica
barcelona['price']=pd.cut(x= barcelona['price'], bins=intervalos, labels= categorias)

ValueError: Bin labels must be one fewer than the number of bin edges

In [62]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelonaf= barcelona.fillna(method='ffill')

<ipython-input-62-c897af8e098f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-62-c897af8e098f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='ffill')


In [63]:
barcelona['price'].isnull().sum()

0

In [107]:
vars_Indep = barcelona[['bedrooms']]
vars_Dep = barcelona['price']

In [108]:
#Redefinimos las variables
x= vars_Indep
y= vars_Dep

In [109]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test, X_train, Y_test, Y_train= train_test_split(x,y, test_size=0.3, random_state=None)

In [110]:
escalar = StandardScaler()

In [111]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [112]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [113]:
#Entrenamos el modelo
algoritmo.fit(X_train, Y_train)

LogisticRegression()

In [114]:
#realizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([136., 136., 136., ..., 146., 136., 136.])

In [115]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test, y_pred)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [116]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score

precision = precision_score(Y_test, y_pred, average="binary", pos_label="barato")
print('Precision del modelo:')
print(precision)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

In [ ]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test, y_pred, average="binary", pos_label="cheap")
print('Sensibilidad del modelo:')
print(sensibilidad)

**2 Categoricas**

In [117]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['review_scores_value'].max()
Min=barcelona['review_scores_value'].min()
Limites= [Min, Max]
Limites

[0.0, 5.0]

In [149]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos1=np.linspace(0.0, 5.0,2)
intervalos1

array([0., 5.])

In [150]:
#Creamos las categorías
categorias1= ["buena", "mala"]

In [151]:
barcelona['review_scores_value']

,review_scores_value
0,4.74
1,4.00
2,4.35
3,4.92
4,4.79
...,...
18918,0.00
18919,0.00
18920,0.00
18921,0.00


In [152]:
#Finalmente creamos las categorías en la columna numérica
barcelona['review_scores_value']=pd.cut(x= barcelona['review_scores_value'], bins=intervalos1, labels= categorias1 )

ValueError: Bin labels must be one fewer than the number of bin edges

In [ ]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelonaf= barcelona.fillna(method='ffill')

In [ ]:
barcelona['review_scores_value'].isnull().sum()

In [122]:
vars_Indep_1 = barcelona[['bedrooms']]
vars_Dep_1 = barcelona['review_scores_value']

In [123]:
#Redefinimos las variables
x= vars_Indep_1
y= vars_Dep_1

In [124]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test1, X_train1, Y_test1, Y_train1= train_test_split(x,y, test_size=0.3, random_state=None)

In [125]:
escalar1 = StandardScaler()

In [126]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train1 = escalar.fit_transform(X_train1)
X_test1 = escalar.transform(X_test1)

In [127]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo1 = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo1.fit(X_train1, Y_train1)

In [128]:
#realizamos una prediccion
y_pred1 = algoritmo1.predict(X_test1)
y_pred1

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test1, y_pred1)
print('Matriz de Confusion')
print(matriz)

In [ ]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score

precision = precision_score(Y_test1, y_pred1, average="binary", pos_label="mala")
print('Precision del modelo:')
print(precision)

In [ ]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud)

In [ ]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test1, y_pred1, average="binary", pos_label="pocos")
print('Sensibilidad del modelo:')
print(sensibilidad)

**3 Categoricas**

In [129]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['accommodates'].max()
Min=barcelona['accommodates'].min()
Limites= [Min, Max]
Limites

[1.0, 7.0]

In [134]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos2=np.linspace(1.0,7.0,3)
intervalos2


array([1., 4., 7.])

In [135]:
#Creamos las categorías
categorias2= ["incomodo", "comodo"]

In [136]:
barcelona['accommodates']

,accommodates
0,2.0
1,6.0
2,3.0
3,3.0
4,5.0
...,...
18918,2.0
18919,4.0
18920,4.0
18921,2.0


In [137]:
#Finalmente creamos las categorías en la columna numérica
barcelona['accommodates']=pd.cut(x= barcelona['accommodates'], bins=intervalos2, labels= categorias2 )


In [138]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelonaf= barcelona.fillna(method='ffill')


<ipython-input-138-2a919a1b957c>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-138-2a919a1b957c>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelonaf= barcelona.fillna(method='ffill')


In [140]:
barcelona['accommodates'].isnull().sum()

0

In [153]:
vars_Indep_2 = barcelona[['bedrooms']]

vars_Dep_2 = barcelona['accommodates']


In [154]:
#Redefinimos las variables
x= vars_Indep_2
y= vars_Dep_2


In [155]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test2, X_train2, Y_test2, Y_train2= train_test_split(x,y, test_size=0.3, random_state=None)


In [156]:
escalar2 = StandardScaler()

In [157]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train2 = escalar.fit_transform(X_train2)
X_test2 = escalar.transform(X_test2)


In [158]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo2 = LogisticRegression()


In [159]:
#Entrenamos el modelo
algoritmo2.fit(X_train2, Y_train2)


LogisticRegression()

In [160]:
#realizamos una prediccion
y_pred2 = algoritmo2.predict(X_test2)
y_pred2


array(['incomodo', 'incomodo', 'incomodo', ..., 'incomodo', 'incomodo',
       'incomodo'], dtype=object)

In [161]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test2, y_pred2)
print('Matriz de Confusion')
print(matriz)


Matriz de Confusion
[[ 284 2470]
 [ 622 9870]]


In [163]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test2, y_pred2, average="binary", pos_label="incomodo")
print('Precision del modelo:')
print(precision)


Precision del modelo:
0.799837925445705


In [164]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test2, y_pred2)
print('Exactitud del modelo:')
print(exactitud)


Exactitud del modelo:
0.7665710403140571


In [166]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test2, y_pred2, average="binary", pos_label="incomodo")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.9407167365611895


**4 categoricas**

In [167]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['review_scores_rating'].max()
Min=barcelona['review_scores_rating'].min()
Limites= [Min, Max]
Limites


[0.0, 5.0]

In [168]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos3=np.linspace(0.0,5.0, 3)
intervalos3


array([0. , 2.5, 5. ])

In [170]:
#Creamos las categorías
categorias3= ["muy mala", "muy buena"]


In [171]:
barcelona['review_scores_rating']

,review_scores_rating
0,4.74
1,4.00
2,4.35
3,4.92
4,4.79
...,...
18918,0.00
18919,0.00
18920,0.00
18921,0.00


In [172]:
#Finalmente creamos las categorías en la columna numérica
barcelona['review_scores_rating']=pd.cut(x= barcelona['review_scores_rating'], bins=intervalos3, labels= categorias3 )


In [177]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelona= barcelona.fillna(method='ffill')


<ipython-input-177-c897af8e098f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-177-c897af8e098f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='ffill')


In [178]:
barcelona['review_scores_rating'].isnull().sum()

0

In [179]:
vars_Indep_3 = barcelona[['bedrooms']]

vars_Dep_3 = barcelona['review_scores_rating']


In [180]:
#Redefinimos las variables
x= vars_Indep_3
y= vars_Dep_3


In [181]:

#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test3, X_train3, Y_test3, Y_train3= train_test_split(x,y, test_size=0.3, random_state=None)


In [182]:
escalar3 = StandardScaler()

In [183]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train3 = escalar.fit_transform(X_train3)
X_test3 = escalar.transform(X_test3)


In [184]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo3 = LogisticRegression()


In [185]:
#Entrenamos el modelo
algoritmo3.fit(X_train3, Y_train3)


LogisticRegression()

In [186]:
#realizamos una prediccion
y_pred3 = algoritmo3.predict(X_test3)
y_pred3


array(['muy buena', 'muy buena', 'muy buena', ..., 'muy buena',
       'muy buena', 'muy buena'], dtype=object)

In [187]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test3, y_pred3)
print('Matriz de Confusion')
print(matriz)


Matriz de Confusion
[[13084     0]
 [  162     0]]


In [188]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test3, y_pred3, average="binary", pos_label="muy mala")
print('Precision del modelo:')
print(precision)


Precision del modelo:
0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [189]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test3, y_pred3)
print('Exactitud del modelo:')
print(exactitud)


Exactitud del modelo:
0.9877698927978258


In [191]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test3, y_pred3, average="binary", pos_label="muy mala")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.0


**5 Categoria**

In [192]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['number_of_reviews'].max()
Min=barcelona['number_of_reviews'].min()
Limites= [Min, Max]
Limites


[0.0, 120.0]

In [193]:
barcelona['number_of_reviews']

,number_of_reviews
0,90.0
1,2.0
2,41.0
3,18.1
4,78.0
...,...
18918,0.0
18919,0.0
18920,0.0
18921,0.0


In [194]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos4=np.linspace(0.0, 120.0, 30)
intervalos4


array([  0.        ,   4.13793103,   8.27586207,  12.4137931 ,
        16.55172414,  20.68965517,  24.82758621,  28.96551724,
        33.10344828,  37.24137931,  41.37931034,  45.51724138,
        49.65517241,  53.79310345,  57.93103448,  62.06896552,
        66.20689655,  70.34482759,  74.48275862,  78.62068966,
        82.75862069,  86.89655172,  91.03448276,  95.17241379,
        99.31034483, 103.44827586, 107.5862069 , 111.72413793,
       115.86206897, 120.        ])

In [195]:
#Creamos las categorías
categorias4= ["pocas", "muchas"]


In [196]:
barcelona['number_of_reviews']

,number_of_reviews
0,90.0
1,2.0
2,41.0
3,18.1
4,78.0
...,...
18918,0.0
18919,0.0
18920,0.0
18921,0.0


In [198]:
#Finalmente creamos las categorías en la columna numérica
barcelona['number_of_reviews']=pd.cut(x= barcelona['number_of_reviews'], bins=intervalos4, labels= categorias4 )


ValueError: Bin labels must be one fewer than the number of bin edges

In [199]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelonfa= barcelona.fillna(method='ffill')


<ipython-input-199-386dee9a1d21>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-199-386dee9a1d21>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelonfa= barcelona.fillna(method='ffill')


In [200]:
barcelona['number_of_reviews'].isnull().sum()

0

In [201]:
vars_Indep_4 = barcelona[['bedrooms']]

vars_Dep_4 = barcelona['number_of_reviews']


In [202]:
#Redefinimos las variables
x= vars_Indep_4
y= vars_Dep_4


In [203]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test4, X_train4, Y_test4, Y_train4= train_test_split(x,y, test_size=0.3, random_state=None)


In [204]:
escalar4 = StandardScaler()

In [205]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train4 = escalar.fit_transform(X_train4)
X_test4 = escalar.transform(X_test4)


In [206]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo4 = LogisticRegression()


In [207]:
#Entrenamos el modelo
algoritmo4.fit(X_train4, Y_train4)


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
#realizamos una prediccion
y_pred4 = algoritmo4.predict(X_test4)
y_pred4


In [ ]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test4, y_pred4)
print('Matriz de Confusion')
print(matriz)


In [ ]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test4, y_pred4, average="binary", pos_label="pocas")
print('Precision del modelo:')
print(precision)


In [ ]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test4, y_pred4)
print('Exactitud del modelo:')
print(exactitud)


In [ ]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test4, y_pred4, average="binary", pos_label="pocas")
print('Sensibilidad del modelo:')
print(sensibilidad)
